In [3]:
import torch
import torch.nn.functional as F

def create_sliding_window_causal_mask(seq_len: int, window_size: int, device: torch.device = None):
    """
    언어 모델을 위한 인과적 슬라이딩 윈도우 어텐션 마스크를 생성하고,
    그 과정을 단계별로 출력하여 이해를 돕습니다.
    
    Args:
        seq_len (int): 전체 시퀀스의 길이입니다.
        window_size (int): 어텐션을 허용할 창의 크기입니다.
        device (torch.device, optional): 텐서를 생성할 디바이스입니다.

    Returns:
        torch.Tensor: 어텐션 계산에 사용될 최종 불리언(boolean) 마스크입니다. 
    """
    print(f"\n--- 마스크 생성 시작 (seq_len={seq_len}, window_size={window_size}) ---")
    
    # 1. 인과성 마스크 (Causal Mask) 생성
    #    미래 토큰을 보지 못하도록 대각선을 포함한 하삼각 행렬을 만듭니다.
    causal_mask = torch.tril(torch.ones((seq_len, seq_len), device=device, dtype=torch.bool), diagonal=0)
    print("\n[1단계] 인과성(Causal) 마스크 (미래 차단):\n", causal_mask.int())

    # 2. 슬라이딩 윈도우 전용 마스크 생성
    #    각 토큰이 자신으로부터 window_size 만큼 떨어진 과거까지만 보도록 제한하는 '밴드' 형태의 마스크를 만듭니다.
    #    (오류 수정: bool 텐서의 뺄셈(-)을 논리적 XOR(^) 연산으로 변경)
    sliding_window_only_mask = torch.tril(torch.ones_like(causal_mask), diagonal=0) ^ torch.tril(torch.ones_like(causal_mask), diagonal=-window_size)
    print("\n[2단계] 슬라이딩 윈도우 전용 마스크 (먼 과거 차단):\n", sliding_window_only_mask.int())
    
    # 3. 최종 마스크 결합
    #    두 조건을 모두 만족하는 최종 마스크를 생성합니다.
    final_mask = causal_mask & sliding_window_only_mask
    print("\n[3단계] 최종 결합된 SWA 마스크:\n", final_mask.int())
    print("-" * 45)
    
    return final_mask


if __name__ == '__main__':
    # --- 설정 ---
    seq_len = 16
    window_size = 4
    d_model = 8
    n_heads = 2
    batch_size = 1

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 디바이스: {device}")

    # --- 마스크 생성 ---
    attention_mask = create_sliding_window_causal_mask(seq_len, window_size, device)
    
    # --- 실제 어텐션 계산에 적용 ---
    print("\n--- 실제 어텐션 계산 예시 ---")
    
    q = torch.randn(batch_size, n_heads, seq_len, d_model // n_heads, device=device)
    k = torch.randn(batch_size, n_heads, seq_len, d_model // n_heads, device=device)
    v = torch.randn(batch_size, n_heads, seq_len, d_model // n_heads, device=device)
    
    print(f"입력 Q, K, V Shape: {q.shape}")
    
    # Full Attention (비교용)
    # 연산량은 seq_len^2에 비례합니다.
    output_full = F.scaled_dot_product_attention(q, k, v, attn_mask=torch.tril(torch.ones(seq_len, seq_len, device=device, dtype=torch.bool)))
    
    # SWA
    # 연산량은 seq_len * window_size에 비례합니다.
    output_swa = F.scaled_dot_product_attention(q, k, v, attn_mask=attention_mask)
    
    print(f"SWA 적용 후 출력 Shape: {output_swa.shape}")
    print("계산이 성공적으로 완료되었습니다.")
    
    # 연산량 비교
    print("\n--- 연산량 비교 ---")
    full_attn_ops = seq_len * seq_len
    swa_ops = seq_len * window_size
    print(f"Full Attention 상대적 연산량: {full_attn_ops}")
    print(f"SWA 상대적 연산량: {swa_ops}")
    print(f"효율성 향상: {full_attn_ops / swa_ops:.2f} 배")

사용 디바이스: cuda

--- 마스크 생성 시작 (seq_len=16, window_size=4) ---

[1단계] 인과성(Causal) 마스크 (미래 차단):
 tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 